In [ ]:
### MinHash Testing
##MIGHT NOT WORK AT TITLE LEVEL (NO ENOUGH HASHESE)

In [398]:
import logging 
logger  = logging.getLogger(__name__)
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
import traceback 
import sys 
import os 
import os.path 
from os import path
import re
import pandas as pd
import urllib
import time 
import random
import string
import binascii
from collections import defaultdict
from text_dedup_tests.MinHash import MinHash
_list = ['i am zack', 'zack is i', 'me and zack are the same','i am -#\$zack','19358-ead84www.g\\']

def clean_sentence(sentence,PUNCTUATION=string.punctuation+"\\\\"):
    sentence = sentence.encode('ascii',errors = 'ignore').decode()
    sentence=re.sub(f'[{PUNCTUATION}]',' ',sentence)
    sentence = re.sub(' {2,}',' ', sentence)
    sentence= sentence.lower().strip()
    return sentence 

pd.set_option('max_colwidth',-1)
    

In [414]:
#load data 
col2check = 'title'
data = pd.read_pickle("C:\\Users\\zjc10\\Desktop\\Projects\\data\\news\\webhose_news\\webhose_df.pickle")
Data = data[~(data[col2check].isna()) & (data[col2check].str.len()>10)].copy()
Data[col2check] = Data[col2check].apply(lambda title: clean_sentence(title))
dtest = Data.head(500).reset_index(drop=True)

In [415]:
PUNCTUATION = re.compile(r"[\.!?,:;]\s|[()]")
MAX_SHINGLE_ID = (2 ** 32) - 1
NEXT_PRIME = 4294967311

DEFAULT_SHINGLE_LEN = 1
DEFAULT_DUPE_SIM_THRESHOLD = 0.05
DEFAULT_NUM_HASHES = 500

#title we want to look for 
docs = dtest['title']
title = pd.Series('disruption in amazons cloud service ripples through internet')
threshold=DEFAULT_DUPE_SIM_THRESHOLD
num_hashes=DEFAULT_NUM_HASHES
shingle_len=DEFAULT_SHINGLE_LEN
docs = docs

In [416]:
#change index of the title we are searching for to be unique
as_list = title.index.tolist()
idx = as_list.index(title.index)
as_list[idx] = 9999999
title.index = as_list

Int64Index([9999999], dtype='int64')

In [427]:
docs = pd.concat([title,docs])

In [428]:
hash_funcs = generate_hash_functions(num_hashes) #50 hash functions
print(hash_funcs[:1]) #list of deafult_num_hashes hash functions 

[<function generate_hash_functions.<locals>.<listcomp>.<lambda>.<locals>.<lambda> at 0x000002C70FB10708>]


In [429]:
shingle_ids = [set(get_shingle_ids(shingle_len, d)) for d in docs]
print(len(shingle_ids),type(shingle_ids))
print(shingle_ids[0]);print([len(x) for x in docs.head(3)])

502 <class 'list'>
{3589173276, 1609338446, 2096316497, 3785202386, 844895956, 3126086714, 3680971804, 3418787229}
[60, 60, 27]


In [430]:
minhash_sigs = [get_minhash_signatures(ids_, hash_funcs) for ids_ in shingle_ids] #120 minhash_sigs each with 50 has values(the lowest)
dup_dict = {}
len([len(x) for x in minhash_sigs])


502

In [431]:
for j in range(1, len(docs)):
    count = sum([minhash_sigs[0][k] == minhash_sigs[j][k] for k in range(0, num_hashes)])
    sim = count / num_hashes
    #print(sim)

    #dup_list.append((j,sim))
    if sim >= threshold:
        dup_list.append(j)
        dup_dict[j] = sim
        
        
idx_to_remove = list(set([val for val in dup_list]))
idx_to_remove = [key for key in sorted(dup_dict, key=lambda x : dup_dict[x])]

print(idx_to_remove)

[3, 7, 18, 35, 73, 83, 119, 179, 193, 234, 235, 371, 385, 5, 443, 57, 99, 117, 324, 454, 186, 23, 60, 76, 85, 114, 155, 382, 373, 396, 115, 483, 133, 42, 218, 181, 240, 244, 174, 397, 197, 34, 407, 317, 362, 250, 409, 56, 171, 356, 123, 33, 59, 1]


In [432]:
dup_dict.keys()
TUP_LIST = [(key,dup_dict[key]) for key in sorted(dup_dict, key=lambda x : dup_dict[x])]
TUP_LIST.sort(key=lambda tup: tup[1], reverse=True)

dtest.loc[[tup[0]-1 for tup in TUP_LIST],'title']

0      brazil espaco cidadania 003                                                                                                       
58     amazon cloud outage disrupts traffic for websites apps                                                                            
32     youtube challenges cable tv with streaming service                                                                                
122    amazon opens book on retail                                                                                                       
355    author interview mandy majors talk a practical approach to cyberparenting and open communication                                  
55     erotic horror music february is women in horror month                                                                             
170    how to add a new user to your nintendo switch                                                                                     
408    27 tips and products for an

In [433]:
[(dtest.loc[tup[0]-1,'title'],tup[1]) for tup in TUP_LIST]


[('brazil espaco cidadania 003', 1.0),
 ('amazon cloud outage disrupts traffic for websites apps', 0.692),
 ('youtube challenges cable tv with streaming service', 0.674),
 ('amazon opens book on retail', 0.244),
 ('author interview mandy majors talk a practical approach to cyberparenting and open communication',
  0.224),
 ('erotic horror music february is women in horror month', 0.126),
 ('how to add a new user to your nintendo switch', 0.126),
 ('27 tips and products for anyone obsessed with french fries', 0.096),
 ('mentos rolls mint pack of 15 for just 7 42 shipped', 0.094),
 ('just your average girlmostly hella a sci fi adventure released today',
  0.092),
 ('amazon buy fox women clothing at upto 93 off', 0.088),
 ('mastering google search operators in 67 easy steps', 0.086),
 ('reply hot deals re amazon trajan 36 50 41 49 csi oos 39 mm', 0.086),
 ('blog review game changer is now live', 0.084),
 ('adding a radar detector dash cam', 0.082),
 ('amazon buy stayfree dry max all night

In [ ]:
for i in range(0, len(docs)):
    print(i)

    for j in range(i + 1, len(docs)):
        count = sum([minhash_sigs[i][k] == minhash_sigs[j][k] for k in range(0, num_hashes)])

        sim = count / num_hashes

        if sim >= threshold:
            self.dupe_dict[i].append(j)

self.idx_to_remove = list(set([idx for idx_list in self.dupe_dict.values() for idx in idx_list]))


In [16]:
def generate_random_coeff(num_coeff, seed=0):
    """Returns a list of random integer coefficients

    Arguments:
        num_coeff {int} -- number of unique random integer coefficients to return

    Keyword Arguments:
        seed {int} -- for testing purposes, seed value to provide to random number generator (default: {0})

    Returns:
        list -- list of random integer coefficients between 0 and the MAX_SHINGLE_ID (2**32 - 1)
    """

    coeff = set()

    if seed != 0:
        random.seed(seed)

    while len(coeff) != num_coeff:
        coeff.add(random.randint(0, MAX_SHINGLE_ID))

    return list(coeff)


In [18]:
def generate_hash_functions(n_hashes):
    """Returns a list of randomly generated hash functions

    Arguments:
        n_hashes {int} -- number of hash functions to return

    Returns:
        list -- randomly generated hash functions as lambdas
    """

    coeff_A = generate_random_coeff(n_hashes)
    coeff_B = generate_random_coeff(n_hashes)
    return [(lambda i: lambda x: sum([coeff_A[i] * x, coeff_B[i]]) % NEXT_PRIME)(i) for i in range(0, n_hashes)]


In [24]:
def get_shingle_ids(size, doc):
    """Returns the shingle ids for a single document

    Arguments:
        size {int} -- length of shingle ids to return
        doc {str} -- string representing document

    Yields:
        int -- shingle ids for a single document
    """

    doc = PUNCTUATION.sub(" ", str(doc))
    tokens = [word for word in doc.split(" ") if word != ""]

    for i in range(0, len(tokens) - size + 1):
        yield (binascii.crc32(bytes(" ".join(tokens[i: i + size]), "utf-8")) & 0xFFFFFFFF)

In [42]:
def get_minhash_signatures(shingle_ids, hash_funcs):
    """Returns the minhash signatures for a single set of shingle ids

    Arguments:
        shingle_ids {list} -- list of shingle ids from a single document
        hash_funcs {list} -- list of randomly generated hash functions

    Returns:
        list -- minimum hash values computed from the shingle ids for each hash function
    """

    return [min([hash_func(id_) for id_ in shingle_ids]) for hash_func in hash_funcs]


In [ ]:
import re
import binascii
import random
from collections import defaultdict

PUNCTUATION = re.compile(r"[\.!?,:;]\s|[()]")
MAX_SHINGLE_ID = (2 ** 32) - 1
NEXT_PRIME = 4294967311

DEFAULT_SHINGLE_LEN = 3
DEFAULT_DUPE_SIM_THRESHOLD = 0.8
DEFAULT_NUM_HASHES = 50


class MinHash:
    """Class for modularizing minhash algorithm

    Arguments:
        docs {list} -- list of strings representing the documents to compare

    Keyword Arguments:
        threshold {int} -- similarity threshold to utilize for flagging duplicates (default: {DEFAULT_DUPE_SIM_THRESHOLD})
        num_hashes {int} -- number of hash functions to utilize (default: {DEFAULT_NUM_HASHES})
        shingle_len {int} -- length of shingles to utilize (default: {DEFAULT_SHINGLE_LEN})

    Attributes:
        docs {list} -- list of strings representing the documents to compare
        hash_funcs {list} -- list of randomly generated hash functions utilized to compute hash values from shingle ids
        shingle_ids {list} -- list of hashed shingles representing the n-grams or shingle tuples from each document
        minhash_sigs {list} -- list of minimum hash values for each document computed from the shingle ids for each hash function
        dupe_dict {collections.defaultdict} -- dictionary storing duplicate document indices for each document index
                                               for which duplicate documents exist
        idx_to_remove {list} -- list of duplicate document indexes
    """

    def __init__(self, docs, threshold=DEFAULT_DUPE_SIM_THRESHOLD, num_hashes=DEFAULT_NUM_HASHES, shingle_len=DEFAULT_SHINGLE_LEN):
        """Initializes MinHash object

        Arguments:
            docs {list} -- list of strings representing the documents to compare

        Keyword Arguments:
            threshold {int} -- similarity threshold to utilize for flagging duplicates (default: {DEFAULT_DUPE_SIM_THRESHOLD})
            num_hashes {int} -- number of hash functions to utilize (default: {DEFAULT_NUM_HASHES})
            shingle_len {int} -- length of shingles to utilize (default: {DEFAULT_SHINGLE_LEN})
        """

        self.docs = docs
        self.hash_funcs = self.generate_hash_functions(num_hashes)
        self.shingle_ids = [set(self.get_shingle_ids(shingle_len, d)) for d in docs]
        self.minhash_sigs = [self.get_minhash_signatures(ids_, self.hash_funcs) for ids_ in self.shingle_ids]
        self.dupe_dict = defaultdict(list)

        for i in range(0, len(self.docs)):
            for j in range(i + 1, len(self.docs)):
                count = sum([self.minhash_sigs[i][k] == self.minhash_sigs[j][k] for k in range(0, num_hashes)])

                sim = count / num_hashes

                if sim >= threshold:
                    self.dupe_dict[i].append(j)

        self.idx_to_remove = list(set([idx for idx_list in self.dupe_dict.values() for idx in idx_list]))

    def generate_random_coeff(self, num_coeff, seed=0):
        """Returns a list of random integer coefficients

        Arguments:
            num_coeff {int} -- number of unique random integer coefficients to return

        Keyword Arguments:
            seed {int} -- for testing purposes, seed value to provide to random number generator (default: {0})

        Returns:
            list -- list of random integer coefficients between 0 and the MAX_SHINGLE_ID (2**32 - 1)
        """

        coeff = set()

        if seed != 0:
            random.seed(seed)

        while len(coeff) != num_coeff:
            coeff.add(random.randint(0, MAX_SHINGLE_ID))

        return list(coeff)

    def generate_hash_functions(self, n_hashes):
        """Returns a list of randomly generated hash functions

        Arguments:
            n_hashes {int} -- number of hash functions to return

        Returns:
            list -- randomly generated hash functions as lambdas
        """

        coeff_A = self.generate_random_coeff(n_hashes)
        coeff_B = self.generate_random_coeff(n_hashes)
        return [(lambda i: lambda x: sum([coeff_A[i] * x, coeff_B[i]]) % NEXT_PRIME)(i) for i in range(0, n_hashes)]

    def get_shingle_ids(self, size, doc):
        """Returns the shingle ids for a single document

        Arguments:
            size {int} -- length of shingle ids to return
            doc {str} -- string representing document

        Yields:
            int -- shingle ids for a single document
        """

        doc = PUNCTUATION.sub(" ", str(doc))
        tokens = [word for word in doc.split(" ") if word != ""]

        for i in range(0, len(tokens) - size + 1):
            yield (binascii.crc32(bytes(" ".join(tokens[i: i + size]), "utf-8")) & 0xFFFFFFFF)

    def get_minhash_signatures(self, shingle_ids, hash_funcs):
        """Returns the minhash signatures for a single set of shingle ids

        Arguments:
            shingle_ids {list} -- list of shingle ids from a single document
            hash_funcs {list} -- list of randomly generated hash functions

        Returns:
            list -- minimum hash values computed from the shingle ids for each hash function
        """

        return [min([hash_func(id_) for id_ in shingle_ids]) for hash_func in hash_funcs]

    def jaccard_similarity(self, idx1, idx2):
        """Return the Jaccard similarity between any two documents
        This statistic acts as the 'ground truth' to which the minhash estimate can be compared

        Arguments:
            idx1 {int} -- index of first document in docs
            idx2 {int} -- index of second document in docs

        Returns:
            int -- Jaccard similarity between any two documents
        """

        s1 = self.shingle_ids[idx1]
        s2 = self.shingle_ids[idx2]
        return len(s1.intersection(s2)) / len(s1.union(s2))


